In [1]:
from google.cloud import bigquery
from datetime import datetime

client = bigquery.Client()

project_id = 'snsdisasteralert-372409'
client = bigquery.Client(project=project_id)

# Insert values in a table
from google.cloud import bigquery
# client = bigquery.Client()

dataset_id = "sns"
# For this sample, the table must already exist and have a defined schema
table_id = "tweets"
table_ref = client.dataset(dataset_id).table(table_id)
table = client.get_table(table_ref)


In [2]:
# Running this code will display the query used to generate your previous job

job = client.get_job('bquxjob_2c59c7e9_185dad4c022') # Job ID inserted based on the query results selected to explore
print(job.query)

SELECT * FROM `snsdisasteralert-372409.sns.data3` 


In [3]:
# Running this code will read results from your previous job

job = client.get_job('bquxjob_2c59c7e9_185dad4c022') # Job ID inserted based on the query results selected to explore
results = job.to_dataframe()
results

,id,author_id,text,like_count,retweet_count,quote_count,followers_count,following_count,f0_,tweet_date,tag,sentiment,virality
0,1565148948263837696,809532994444591104,#Serena Williams. \n\nThat’s it. That’s the tw...,20,1990,211,353230,990,0.665685,2022-09-01 00:00:00+00:00,twitter,neutral,0
1,1578859624648036352,1083283217141170176,Nicki Minaj got 14+ of her songs trending on t...,17,3030,384,9183,2978,0.472328,2022-10-08 00:00:00+00:00,twitter,neutral,0
2,1577460057771384832,1200616796295847936,Elon's first day at Twitter? https://t.co/YRkl...,107,5668,832,1065567,2388,0.623323,2022-10-05 00:00:00+00:00,twitter,neutral,0
3,1577093985515945984,42987732,Jamaal Williams somehow found my tweet from Sa...,3,2989,70,178068,1062,0.550796,2022-10-04 00:00:00+00:00,twitter,neutral,0
4,1575730292756660224,906362637264478208,(trans)\n-they called MileApo as the hottest y...,122,2750,1495,19257,258,0.485236,2022-09-30 00:00:00+00:00,twitter,neutral,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
218210,1536791997176074241,237941903,AWS is now recruiting for some open roles in L...,17,1555,765,5025,481,0.595576,2022-06-14 00:00:00+00:00,amazon,positive,2337
218211,1494317805826498564,180740221,#ContestAlert\nThe scarier the better! 😨😲 The ...,12,342,268,22610,19,0.421802,2022-02-17 00:00:00+00:00,amazon,positive,622
218212,1549122392986750976,3153323480,i stopped at the kettleman city supercharger &...,6,911,50,21414,918,1.036286,2022-07-18 00:00:00+00:00,tesla,positive,967
218213,1477725409542348813,39568716,Watching the darts but how big is the Amazon E...,0,494,9,1183,1410,0.744462,2022-01-02 00:00:00+00:00,amazon,positive,503


In [4]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle


# 'Emoji_Dict.p'- download link https://drive.google.com/open?id=1G1vIkkbqPBYPKHcQ8qy0G2zkoab2Qv4v
with open('Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

def convert_emojis_to_word(text):
    for emot in Emoji_Dict:
        text = re.sub(r'('+emot+')', "_".join(Emoji_Dict[emot].replace(",","").replace(":","").split()), text)
    return text

# 'Emoticon_Dict.p'- download link https://drive.google.com/open?id=1HDpafp97gCl9xZTQWMgP2kKK_NuhENlE
with open('Emoticon_Dict.p', 'rb') as fp:
    Emoticon_Dict = pickle.load(fp)

def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in Emoticon_Dict) + u')')
    return emoticon_pattern.sub(r'', text)
    

# Initialize objects
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def preprocess(text):
    text=convert_emojis_to_word(text)
    text=remove_emoticons(text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stop words
    tokens = [token for token in tokens if token not in stop_words]
    # Perform stemming or lemmatization
    tokens = [ps.stem(token) for token in tokens]
    # Identify and process Emoji
    tokens = [token for token in tokens if not re.search(':[a-z_]+:', token)]
    # Identify and process URLs and image links
    tokens = [token for token in tokens if re.sub(r'http\S+', '', token)]
    return " ".join(tokens)

# Example usage
text = "This is an example tweet with a 😃 emoji, a link to an image https://example.com/image.jpg and a URL http://www.example.com"
print(preprocess(text))


thi exampl tweet smiling_face_with_open_mouth emoji , link imag url


In [5]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from afinn import Afinn

# Load your dataset of tweets into a Pandas dataframe
tweets_df = results

# Create a VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

afinn=Afinn()

# Define a function to perform sentiment analysis on a single tweet
def analyze_sentiment(tweet):
    # tweet = preprocess(tweet)
    # Get the sentiment scores for the tweet text
    sentiment_scores = analyzer.polarity_scores(tweet)
    # Extract the compound sentiment score
    compound_score = sentiment_scores['compound']
    # Map the compound score to a sentiment label (positive, neutral, negative)
    if compound_score >= 0.05:
        sentiment_label = 'positive'
    elif compound_score <= -0.05:
        sentiment_label = 'negative'
    else:
        sentiment_label = 'neutral'
    return sentiment_label
# Define a function to perform sentiment analysis on a single tweet
def analyze_sentiment2(tweet):
    # tweet = preprocess(tweet)
    # Get the sentiment scores for the tweet text
    sentiment_scores = afinn.score(tweet)
    # Extract the compound sentiment score
    # compound_score = sentiment_scores['compound']
    # Map the compound score to a sentiment label (positive, neutral, negative)
    if sentiment_scores >  0:
        sentiment_label = 'positive'
    elif sentiment_scores < 0:
        sentiment_label = 'negative'
    else:
        sentiment_label = 'neutral'
    return sentiment_label

# Apply the sentiment analysis function to each tweet in the dataset
tweets_df['text_clean'] = tweets_df['text'].apply(preprocess)
tweets_df['sentiments'] = tweets_df['text_clean'].apply(analyze_sentiment)
tweets_df['sentimentss'] = tweets_df['text_clean'].apply(analyze_sentiment2)

In [6]:
import pandas as pd

# Load your dataset into a Pandas dataframe
df =tweets_df

# Compare the sentiment labels in the three columns
comparison_1 = df['sentiment'] == df['sentiments']
comparison_2 = df['sentiment'] == df['sentimentss']
comparison_3 = df['sentiments'] == df['sentimentss']

# Calculate the percentage of similarity
similarity_1 = comparison_1.mean() * 100
similarity_2 = comparison_2.mean() * 100
similarity_3 = comparison_3.mean() * 100

print(f"The percentage of similarity between sentiment and sentiments is {similarity_1:.2f}%")
print(f"The percentage of similarity between sentiment and sentiments_2 is {similarity_2:.2f}%")
print(f"The percentage of similarity between sentiments and sentiments_2 is {similarity_3:.2f}%")


The percentage of similarity between sentiment and sentiments is 80.61%
The percentage of similarity between sentiment and sentiments_2 is 73.84%
The percentage of similarity between sentiments and sentiments_2 is 85.74%


In [7]:
tweets_df.to_csv('datasent.csv') 

In [20]:
tweets_df = pd.read_csv('azeaezf.csv')
newdf = tweets_df.drop(['sentiment', 'sentiments', 'Unnamed: 0'], axis=1)
newdf = newdf.rename(columns={'sentimentss': 'sentiment'})
newdf = newdf.rename(columns={'f0_': 'jaccard'})


In [22]:
import pandas_gbq
project_id = 'snsdisasteralert-372409'
table_id='sns.data4'
pandas_gbq.to_gbq(newdf, table_id, project_id=project_id)

100%|██████████| 1/1 [00:00<?, ?it/s]
